In [ ]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np

#import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import  f1_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from catboost import CatBoostClassifier

import pymorphy2
from string import punctuation
import re

import os
import time

In [3]:
pth1 = '/ML/datasets/toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1, index_col=[0], parse_dates=[0])
elif os.path.exists(pth2):
    data = pd.read_csv(pth2, index_col=[0], parse_dates=[0])
else:
    print('Something is wrong')
    
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


In [4]:
%%time

stop_words = stopwords.words('english')
stop_words.extend(['and', 'im'])

lemma = pymorphy2.MorphAnalyzer()

def preprocess_text(doc):
    
    text = ' '.join([i for i in doc.lower().split() if i not in stop_words])
    text = ''.join(ch for ch in text if ch not in punctuation)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    #text = " ".join([lemma.parse(word)[0].normal_form for word in text.split(' ')])
    return text

data['lemma'] = [preprocess_text(t) for t in data['text']]

CPU times: total: 41.4 s
Wall time: 2min 53s


In [5]:
import torch
import transformers 

ModuleNotFoundError: No module named 'transformers'

In [ ]:
tokenizer = transformers.BertTokenizer(
vocab_file='/datasets/ds_bert/vocab.txt') 

In [ ]:
data['RuBERT'] = tokenizer.encode(data['lemma'], add_special_tokens=True) 

In [ ]:
data

In [ ]:
features = data['lemma']
target = data['toxic']


features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=12345, stratify=target)

print(features_train.shape, target_train.shape, features_test.shape, target_test.shape)

In [17]:
def upsample(X, y, repeat):
    features_zeros = X[y == 0]
    features_ones = X[y == 1]
    target_zeros = y[y == 0]
    target_ones = y[y == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled.reset_index(drop=True), target_upsampled.reset_index(drop=True)


features_upsampled, target_upsampled = upsample(features, target, 9)

In [20]:
count_tf_idf = TfidfVectorizer() 
features_train = count_tf_idf.fit_transform(features_train) 
features_test = count_tf_idf.transform(features_test) 